# CiTree Scraper

[CiTree](https://citree.de/) stellt Informationen über verschiedene Baumarten bereit. Um diese Daten zu verarbeiten, müssen wir sie einmalig herunterladen und aufbereiten. Da wir keinen Datenbank-Dump erhalten, haben wir die Daten mit diesem Notebook heruntergeladen und aufbereitet.

## Download der Baumarten

Da CiTree keine API bereitstellt, um strukturierte Daten im JSON-Format abzurufen, müssen wir diese Daten selbst aufbereiten. Hierfür laden wir im ersten Schritt die Daten herunter und legen sie lokal ab. Hierbei handelt es sich um partielle HTML-Dateien, die auf der Webseite angezeigt werden.

## Lizenzen

Die Inhalte der Website unterliegen der [Creative-Commons-Lizenz BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/deed.de). Das beduetet, wir dürfen sie nichtkommerziell weiterverwenden, aber mit Namensnennung.

In [ ]:
from tqdm import tqdm
import requests
import os

treeId = 1

image_urls = set()

if not os.path.exists("data"):
    os.makedirs("data")

for treeId in tqdm(range(1, 375)):
    if os.path.exists(f"data/{treeId}.html"):
        continue
    URL = f"https://citree.de/AJAX/getSteckbrief.php?ArtInfos=%7B%22ID%22%3A{treeId}%2C%22matched%22%3A%5B44%5D%2C%22unmatched%22%3A%5B0%5D%7D&language=de"
    response = requests.get(URL)
    with open(f"data/{treeId}.html", "w") as f:
        f.write(response.text)

100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


## Parsen der Attribute und Download von Bildern

Im ersten Schritt müssen wir die verfügbaren Attribute der Bäume ermitteln. Hierfür iterieren wir einmal über alle heruntergeladenen Baumarten und suchen alle Attribute aus den HTML-Tabellen heraus.

Außerdem werden in diesem Zuge auch die Bilder der jeweiligen Bäume in der höchsten verfügbaren Qualität heruntergeladen.

In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import os

attributes = {}

if not os.path.exists("images"):
    os.makedirs("images")

for tree_id in tqdm(range(1, 366)):

    with open(f"data/{tree_id}.html", "r") as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    
    tree_image_urls = [i.get('src') for i in soup.find_all('img') if 'img/icons/ic_close_dark.png' not in i.get('src')]
    for url in tree_image_urls:
        if os.path.exists(f"images/{url.split('/')[-1]}"):
            continue
        large_image_url = url.replace("/640/", "/1024/")
        download_url = f"https://citree.de/{large_image_url}"
        response = requests.get(download_url)
        if response.status_code == 200:
            with open(f"images/{large_image_url.split('/')[-1]}", "wb") as f:
                f.write(response.content)

    headlines = soup.find_all('h4')
    for h in headlines:
        # print(h.text.strip())

        attribute_name = h.text.strip()

        if attribute_name not in attributes:
            attributes[attribute_name] = {}

        table = h.next_sibling
        for row in table.find_all('tr'):
            if row.find('td').text.strip() not in attributes[attribute_name]:
                attributes[attribute_name][row.find('td').text.strip()] = row['title']

for a in attributes:
    print(a)
    for b in attributes[a]:
        print(f"  - {b}: {attributes[a][b]}")


100%|██████████| 2/2 [00:00<00:00, 112.29it/s]

Klimabedingungen
  - Lichtanspruch: Abschattung des Baumstandortes durch benachbarte Bäume oder Gebäude
  - Trockenheitstoleranz: Gefährdung durch Luft-und Bodentrockenheit
  - Hitzeverträglichkeit: Gefährdung durch Hitzebelastung
  - Spätfrosttoleranz: Gebiete die als spätfrostgefährdet gelten sind Senken, Flussnähe, Freiflächen
  - Winterhärtezone: Einteilung der Klimaregionen anhand der durchschnittlich kältesten Jahrestemperatur
Bodenbedingungen
  - pH-Wert: pH-Wert am Pflanzstandort
  - Bodenverdichtungstoleranz: Risiko des Auftretens von Bodenverdichtung
  - Staunässetoleranz: Risiko des Auftretens von Staunässe
  - Salzverträglichkeit: Wie hoch ist die Salzbelastung
  - Bodenfeuchtetoleranz: Bodenfeuchtigkeit am Standort
  - Bodeneigenschaft: Substrat auf Pflanzenstandort
  - Gründigkeit: Wie tief wurzelt die Baumart
Natürliche Verbreitung
  - Neophyt: Keine heimische Pflanze (wurde nach 1492 eingeführt)
  - Herkunft: 
  - Natürlicher Lebensraum: Natürlicher Lebensraum der Pflan

## Parsen der Baumdaten

Anhand der ermittelten Attribute können wir die Baumdaten nun parsen.

In [ ]:
from bs4 import BeautifulSoup
import re

trees = []

for tree_id in tqdm(range(1, 366)):
    tree_information = {}

    tree_information['id'] = tree_id
    
    trivial_name_tag = soup.find('span', class_='name-trivial')
    tree_information['name_trivial'] = trivial_name_tag.text.strip() if trivial_name_tag else None

    botanic_name_tag = soup.find('span', class_='name-botanic')
    tree_information['name_botanic'] = botanic_name_tag.text.strip() if botanic_name_tag else None

    strain_tag = soup.find('div', class_='strain')
    tree_information['strain'] = strain_tag.text.strip().replace("Sorte: ", "") if strain_tag else None

    with open(f"data/{tree_id}.html", "r") as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    tree_information['id'] = tree_id
    
    trivial_name_tag = soup.find('span', class_='name-trivial')
    tree_information['name_trivial'] = trivial_name_tag.text.strip() if trivial_name_tag else None

    botanic_name_tag = soup.find('span', class_='name-botanic')
    tree_information['name_botanic'] = botanic_name_tag.text.strip() if botanic_name_tag else None

    strain_tag = soup.find('div', class_='strain')
    tree_information['strain'] = strain_tag.text.strip().replace("Sorte: ", "").strip() if strain_tag else None

    tree_image_urls = [a.get('href').replace("img/1024/", "") for a in [i.parent for i in soup.find_all('img', class_="img-responsive") if 'img/icons/ic_close_dark.png' not in i.get('src')]]
    tree_information['images'] = tree_image_urls

    for attribute_group in attributes:
        for attribute in attributes[attribute_group]:

            value = soup.find('td', string=re.compile(attribute)).find_next_sibling('td').text.strip()
            if value == "Nein":
                value = False
            elif value == "Ja":
                value = True
            elif value == "keine Information":
                value = None

            if attribute in ["Astbruchgefahr", "Blattform", "Blütenfarbe", "Blütenstand", "Bodeneigenschaft", "Bodenfeuchtetoleranz", "Bodenverdichtungstoleranz", "Fruchtfarbe", "Fruchtform", "Gründigkeit", "Herbstfärbung", "Herkunft", "Hitzeverträglichkeit", "Kronendurchlässigkeit", "Kronenform", "Lichtanspruch", "name_botanic", "name_trivial", "Natürlicher Lebensraum", "Salzverträglichkeit", "Spätfrosttoleranz", "Trockenheitstoleranz", "Wuchsform", "Wuchsgeschwindigkeit", "Wuchsrichtung"] and type(value) == str:
                value = value.split(", ")
                
            tree_information[attribute] = value

    trees.append(tree_information)

100%|██████████| 2/2 [00:00<00:00, 37.72it/s]


## Daten als JSON ablegen

Die eingelesenen Daten wollen wir für eine mögliche Weiterverarbeitung als JSON ablegen. Hierfür können wir das erzeugte Array aus dem vorigen Schritt ablegen.

In [7]:
import json

with open("tree-information.json", "w") as f:
    json.dump({
        "attributes": attributes,
        "trees": trees
    }, f, indent=4)